## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


In [3]:
(x, y), (x_test, y_test)=mnist_dataset()
print(x.shape, y.shape, x.min(), x.max())

(60000, 28, 28) (60000,) 0.0 1.0


## 建立模型

In [7]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 100],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,initial_value=tf.random.uniform(shape=[100, 10],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        reshaped_input = tf.reshape(x, shape=[-1, 28*28])
        hidden_layer = tf.tanh(tf.matmul(reshaped_input, self.W1) + self.b1)
        logits = tf.matmul(hidden_layer, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [8]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.1130748 ; accuracy 0.37276667
epoch 1 : loss 2.1086671 ; accuracy 0.37986666
epoch 2 : loss 2.104272 ; accuracy 0.38661668
epoch 3 : loss 2.0998883 ; accuracy 0.39375
epoch 4 : loss 2.0955164 ; accuracy 0.40048334
epoch 5 : loss 2.0911558 ; accuracy 0.40643334
epoch 6 : loss 2.086806 ; accuracy 0.41241667
epoch 7 : loss 2.082467 ; accuracy 0.41915
epoch 8 : loss 2.0781388 ; accuracy 0.42556667
epoch 9 : loss 2.0738206 ; accuracy 0.4318
epoch 10 : loss 2.0695128 ; accuracy 0.43725
epoch 11 : loss 2.0652144 ; accuracy 0.44281667
epoch 12 : loss 2.0609257 ; accuracy 0.44873333
epoch 13 : loss 2.0566463 ; accuracy 0.45436665
epoch 14 : loss 2.0523763 ; accuracy 0.45941666
epoch 15 : loss 2.0481148 ; accuracy 0.4654
epoch 16 : loss 2.0438623 ; accuracy 0.47066668
epoch 17 : loss 2.0396185 ; accuracy 0.47598332
epoch 18 : loss 2.035383 ; accuracy 0.48146668
epoch 19 : loss 2.0311556 ; accuracy 0.48568332
epoch 20 : loss 2.0269365 ; accuracy 0.49056667
epoch 21 : loss 2.02272